In [15]:
!pip install chromadb
!pip install datasets
!pip install konlpy
!pip install sentence-transformers

/Users/jb/miniconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=16784) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 45.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.4/583.4 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 44.1 MB/s eta 0:00:00


In [21]:
# declare name of the collection
COLLECTION_NAME = "QuestionAnswering"
QUERY_SENTENCE = "사이보그가 뭐야?"
NUM_OF_DATA = 200
WEIGHT_OF_VECTOR_IN_HYBRID = 0.5

#ChromaDB
EMBEDDING_MODEL = "BAAI/bge-m3"
DISTANCE_METHOD = "l2" # default is l2

In [22]:
import pandas as pd
from datasets import load_dataset

DATA_SET    = "beomi/KoAlpaca-v1.1a"
DATA_FILE   = "./data/KoAlpaca-train.csv"

load_data = load_dataset(DATA_SET, split="train")
load_data.to_csv(DATA_FILE)
csv_data = pd.read_csv(DATA_FILE)
csv_data.head(5)

Creating CSV from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

,instruction,output,url
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...,https://kin.naver.com/qna/detail.naver?d1id=11...
1,스웨터의 유래는 어디에서 시작되었나요?,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,https://kin.naver.com/qna/detail.naver?d1id=11...
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",https://kin.naver.com/qna/detail.naver?d1id=11...
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,https://kin.naver.com/qna/detail.naver?d1id=5&...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...","'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...",https://kin.naver.com/qna/detail.naver?d1id=11...


In [ ]:
# this is for Korean tokenization

# from konlpy.tag import Okt
# from konlpy.tag import Kkma
#
# okt = Okt()
# kkma = Kkma()
#
# print('OKT 형태소 분석 :',okt.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
# print('OKT 품사 태깅 :',okt.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
# print('OKT 명사 추출 :',okt.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
#
# print('꼬꼬마 형태소 분석 :',kkma.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
# print('꼬꼬마 품사 태깅 :',kkma.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
# print('꼬꼬마 명사 추출 :',kkma.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))

In [26]:
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

# embedding model load
model = SentenceTransformer(
    model_name_or_path=EMBEDDING_MODEL
)

# default_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBEDDING_MODEL)



In [27]:
import chromadb
# from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.PersistentClient() # store all data locally in a directory on your machine

# client = chromadb.HttpClient(
#     host="localhost",
#     port=8000,
#     ssl=False,
#     headers=None,
#     settings=Settings(),
#     tenant=DEFAULT_TENANT,
#     database=DEFAULT_DATABASE,
# )

In [28]:
# Drop the collection
client.delete_collection(name=COLLECTION_NAME)

collection = client.get_or_create_collection(
    name=COLLECTION_NAME,
    # metadata={"hnsw:space": DISTANCE_METHOD}
)

In [29]:
from tqdm import tqdm

# Fetch the first 200 rows from csv_data
data_to_insert = csv_data.head(min(NUM_OF_DATA, len(csv_data)))

# ready for data
# order : index, embedding, metadata, document, uri
ids = list()
embeddings = list()
metadatas = list()

for index, row in tqdm(data_to_insert.iterrows()):
    instruction = row.instruction
    output = row.output
    url = row.url

    embedding = model.encode(instruction, normalize_embeddings=True)
    # Concatenate instruction and output
    # combined_text = instruction + " " + output
    # embedding = model.encode(combined_text, normalize_embeddings=True)

    metadata = {
        "instruction": instruction,
        "output": output,
        "url": url
    }

    ids.append(str(index))
    embeddings.append(embedding)
    metadatas.append(metadata)


# set the size to save once
CHUNK_SIZE = 4096
total_index = len(embeddings) // CHUNK_SIZE + 1
# convert numpy arrary to list
embeddings = [ embedding.tolist() for embedding in tqdm(embeddings) ]

for index in tqdm(range(total_index)):
    # set 'start' and 'end' index
    start_idx = index * CHUNK_SIZE
    end_idx = (index + 1) * CHUNK_SIZE

    # save data from selected indexes
    chunk_ids = ids[start_idx:end_idx]
    chunk_embeddings = embeddings[start_idx:end_idx]
    chunk_metadatas = metadatas[start_idx:end_idx]
    # add chunk data in collection
    collection.add(
        ids=chunk_ids,
        embeddings=chunk_embeddings,
        metadatas=chunk_metadatas,
    )


200it [08:27,  2.54s/it]
100%|██████████| 1/1 [00:01<00:00,  1.12s/it]


In [30]:
from pprint import pprint

# semantic search
result = collection.query(
    query_embeddings=model.encode(
        QUERY_SENTENCE,
        normalize_embeddings=True
    ).tolist(),
    n_results=3
)

pprint(result)

{'data': None,
 'distances': [[0.5244351029396057, 0.9750370383262634, 1.1075068712234497]],
 'documents': [[None, None, None]],
 'embeddings': None,
 'ids': [['4', '169', '119']],
 'included': [<IncludeEnum.distances: 'distances'>,
              <IncludeEnum.documents: 'documents'>,
              <IncludeEnum.metadatas: 'metadatas'>],
 'metadatas': [[{'instruction': "'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 "
                                '알고 싶습니다.',
                 'output': "'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로운 "
                           '개념을 나타냅니다. 주로 공상과학영화에서 등장하며, 1985년 페미니스트인 도나 해러웨이가 '
                           "발표한 '사이보그를 위한 선언문'으로 인해 사이보그는 성차별 사회를 극복하는 사회정치적 "
                           '상징으로 더욱 주목받게 되었습니다. 사이보그는 생물체와 기술적 개입이 결합된 다양한 종류가 '
                           '있으며, 이제는 인공장기를 받은 사람, 예방접종을 받은 사람, 또는 사람과 기계가 결합된 '
                           '베리칩과 같은 기술적 개입을 받은 사람들까지 우리 주변에서 만날 수 있습니다. \n'
                           '\n'
                           '또한, 